In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import librosa
import math
%matplotlib inline

In [4]:
s, sr_s=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr_x=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
print(S.shape)
print(X.shape)
S_abs = np.abs(S)
X_abs = np.abs(X)

(513, 2459)
(513, 2459)


In [95]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 64

# https://riptutorial.com/tensorflow/example/19385/basic-example
# https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2

# input of length -1, and dimension 513.
x = tf.placeholder(tf.float32, [BATCH_SIZE, 1, 513])
y_ = tf.placeholder(tf.float32, [BATCH_SIZE, 1, 513])

# filter with width 3, and we take 513 channels as input, and output also 100 channels.
filters = tf.Variable(tf.random_normal([3,513,100]))

conv1 = tf.nn.conv1d(
    x,            
    filters=filters,
    padding="SAME",
    stride=1)

l1 = tf.layers.dense(conv1, units=200,  activation=tf.nn.relu)
l2 = tf.layers.dense(l1, units=200,  activation=tf.nn.relu)
y = tf.layers.dense(l2, units=513, activation=tf.nn.relu)

loss = tf.losses.mean_pairwise_squared_error(labels=y_, predictions=y)
# train_step = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

sess=tf.Session()
tf.global_variables_initializer().run(session=sess)

train_data = tf.data.Dataset.from_tensor_slices(tf.constant(X_abs.T[:]))
label_data = tf.data.Dataset.from_tensor_slices(tf.constant(S_abs.T[:]))
zip_data = tf.data.Dataset.zip((train_data, label_data))

batch_data = zip_data.batch(BATCH_SIZE)
batch_data = batch_data.repeat()
iterator = batch_data.make_one_shot_iterator()
next_batch = iterator.get_next()

In [ ]:
TRAIN_STEPS = 50000
for i in range(TRAIN_STEPS+1):
    batch = sess.run(next_batch)
    _, loss_value = sess.run((train_step, loss), feed_dict={x: batch[0], y_: batch[1]})
    if i% 2000 == 0:
        print('Training Step:' + str(i) + '  Loss =  ' + str(loss_value))

In [ ]:
def denoise_sound(input_file_name, output_file_name):
    sn, sr=librosa.load(input_file_name, sr=None)
    testX=librosa.stft(sn, n_fft=1024, hop_length=512)
    testX_abs = np.abs(testX)
    S_test_abs = sess.run(y, feed_dict={x: testX_abs.T}).T
    ratio = (testX / testX_abs)
    Sh = np.multiply(ratio, S_test_abs)
    librosa.output.write_wav(output_file_name, librosa.istft(Sh, hop_length=512), sr)
    

In [ ]:
denoise_sound('data/test_x_01.wav', 'recover_01.wav')
denoise_sound('data/test_x_02.wav', 'recover_02.wav')